# Folium Project

## Description

### Installation Packages

In [89]:
%pip install pandas sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [90]:
%pip install pyodbc

In [91]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


### Code

#### Preparation

In [114]:
import pyodbc
import pandas as pd

# Définition ses informations de connexion
# Aller dans SSMIS -> clique droit sur AdventureWorks2022 -> Properties et récupérer le nom du server et de la database
server = 'LAPTOP-SQ0G8B4V\SQLEXPRESS'
database = 'AdventureWorks2022'

# Établir la connexion
conn = pyodbc.connect(
    f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Windows Authentication"
)


In [135]:
# Exécutation des requêtes SQL et traitez les résultats avec pandas
query = "SELECT V.[Name], A.AddressID, A.AddressLine1, A.AddressLine2, A.City, A.PostalCode, A.SpatialLocation.Lat AS Latitude ,A.SpatialLocation.Long AS Longitude FROM Purchasing.Vendor V INNER JOIN Person.BusinessEntityAddress BEA ON V.BusinessEntityID = BEA.BusinessEntityID INNER JOIN Person.Address A ON BEA.AddressID = A.AddressID"
data = pd.read_sql(query, conn)


C:\Users\veyss\AppData\Local\Temp\ipykernel_79616\1170166061.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


In [136]:
data

,Name,AddressID,AddressLine1,AddressLine2,City,PostalCode,Latitude,Longitude
0,Australia Bike Retailer,309,28 San Marino Ct.,None,Bellingham,98225,48.652995,-122.180922
1,Allenson Cycles,303,4659 Montoya,None,Altadena,91001,34.161374,-117.996806
2,Advanced Bicycles,302,7995 Edwards Ave.,None,Lynnwood,98036,47.808117,-122.317889
3,"Trikes, Inc.",397,90 Sunny Ave,None,Berkeley,94704,37.886476,-122.167269
4,Morgan Bike Accessories,376,9098 Story Lane,None,Albany,12210,42.643826,-73.645069
...,...,...,...,...,...,...,...,...
99,Bloomington Multisport,314,218 Fall Creek Road,None,West Covina,91791,34.048017,-118.069866
100,Carlson Specialties,318,2313 B Southampton Rd,None,Missoula,59801,46.790615,-113.976004
101,"Compete, Inc.",325,9491 Toyon Dr,None,Dallas,75201,32.786195,-96.926752
102,Chicago City Saddles,320,3 Gehringer Drive,None,Daly City,94015,37.634691,-122.581953


#### Folium 

In [144]:
import binascii
import json
import folium
from folium.plugins import HeatMap

In [138]:
data

,Name,AddressID,AddressLine1,AddressLine2,City,PostalCode,Latitude,Longitude
0,Australia Bike Retailer,309,28 San Marino Ct.,None,Bellingham,98225,48.652995,-122.180922
1,Allenson Cycles,303,4659 Montoya,None,Altadena,91001,34.161374,-117.996806
2,Advanced Bicycles,302,7995 Edwards Ave.,None,Lynnwood,98036,47.808117,-122.317889
3,"Trikes, Inc.",397,90 Sunny Ave,None,Berkeley,94704,37.886476,-122.167269
4,Morgan Bike Accessories,376,9098 Story Lane,None,Albany,12210,42.643826,-73.645069
...,...,...,...,...,...,...,...,...
99,Bloomington Multisport,314,218 Fall Creek Road,None,West Covina,91791,34.048017,-118.069866
100,Carlson Specialties,318,2313 B Southampton Rd,None,Missoula,59801,46.790615,-113.976004
101,"Compete, Inc.",325,9491 Toyon Dr,None,Dallas,75201,32.786195,-96.926752
102,Chicago City Saddles,320,3 Gehringer Drive,None,Daly City,94015,37.634691,-122.581953


In [139]:
# Création d'une carte Folium 
m = folium.Map(location = [40, -95], zoom_start = 4)
display(m)

In [140]:
# Ajout des marqueurs à la carte
for index, row in data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row[['Name','City']],
        icon=folium.Icon(color='red'),

    ).add_to(m)

# Affichez la carte 
m

In [145]:
# Création d'une liste de points de données pour la HeatMap
heat_data = [[row['Latitude'], row['Longitude']] for index, row in data.iterrows()]

# Création de la HeatMap et ajout à la carte Folium
HeatMap(heat_data).add_to(m)

m

#### Fermeture de la connection avec AdventureWorks2022

In [146]:
# Fermeture de la connexion
conn.close()